<a href="https://colab.research.google.com/github/DumontHenry/exercise_PySpark/blob/main/PySpark_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark==3.4.1
!pip install findspark==2.0.1

In [5]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark 101 Exercises").getOrCreate()
print(spark.version)


3.4.1


In [6]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [9]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# Applying orderBy() and monotonically_increasing_id()
window_spec = Window.orderBy(monotonically_increasing_id())

# Add a new column "row_number" using row_number() over the specified window
result_df = df.withColumn("Index", row_number().over(window_spec)-1)

# Show the result
result_df.show()

+-------+-----+-----+
|   Name|Value|Index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



In [10]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]

In [11]:
# Create an RDD from the lists and convert it to a DataFrame
rdd = spark.sparkContext.parallelize(list(zip(list1, list2)))
df = rdd.toDF(["Column1", "Column2"])

# Show the DataFrame
df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
|      d|      4|
+-------+-------+



In [12]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [13]:
sc = spark.sparkContext

rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

result_rdd = rdd_A.subtract(rdd_B)
# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3]


In [14]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [17]:
sc = spark.sparkContext

rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

result_rdd_A = rdd_A.subtract(rdd_B)
result_rdd_B = rdd_B.subtract(rdd_A)

result_rdd = result_rdd_A.union(result_rdd_B)
# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3, 8, 6, 7]


In [18]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+



In [20]:
quantiles = df.approxQuantile("Age", [0.0, 0.25, 0.5, 0.75, 1.0], 0.01)
print(quantiles)
print("Min: ", quantiles[0])
print("25th percentile: ", quantiles[1])
print("Median: ", quantiles[2])
print("75th percentile: ", quantiles[3])
print("Max: ", quantiles[4])

[10.0, 20.0, 30.0, 50.0, 86.0]
Min:  10.0
25th percentile:  20.0
Median:  30.0
75th percentile:  50.0
Max:  86.0


In [21]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [26]:
df.select('name', 'job').distinct().collect()
####
df.groupby('job').count().show()
df.groupby('name').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
| Engineer|    4|
|   Doctor|    1|
+---------+-----+

+----+-----+
|name|count|
+----+-----+
|Mary|    1|
|John|    2|
| Bob|    3|
| Sam|    1|
+----+-----+



In [27]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [32]:
from pyspark.sql.functions import col, when

# Get the top 2 most frequent jobs
top_2_jobs= df.groupby('job').count().orderBy('count', ascending=False).limit(2).select('job').rdd.flatMap(lambda x: x).collect()
# Replace all but the top 2 most frequent jobs with 'Other'
df = df.withColumn('job', when(col('job').isin(top_2_jobs), col('job')).otherwise('Other'))
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



In [43]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|null|
|   B| null| 123|
|   B|    3| 456|
|   D| null|null|
+----+-----+----+



In [42]:
df_1 = df.dropna(subset=['Value'], how='all')
df_1.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|null|
|   B|    3| 456|
+----+-----+----+



In [44]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+



In [45]:
for old_name, new_name in zip(old_names, new_names):
  df = df.withColumnRenamed(old_name, new_name )
df.show()

+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



In [46]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))

df.show(5)

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows



In [47]:
num_buckets = 10  # Number of buckets
quantiles = df.stat.approxQuantile("values",[i/num_buckets for i in range(num_buckets+1)], 0.01)

# Create the Bucketizer
bucketizer = Bucketizer(splits=quantiles, inputCol="values", outputCol="buckets")

# Apply the Bucketizer
df_buck = bucketizer.transform(df)

#Frequency table
df_buck.groupBy("buckets").count().show()

# Show the original and bucketed values
df_buck.show(5)


+-------+-----+
|buckets|count|
+-------+-----+
|    8.0|   10|
|    0.0|    8|
|    7.0|   10|
|    1.0|   10|
|    4.0|   10|
|    3.0|   10|
|    2.0|   10|
|    6.0|   10|
|    5.0|   10|
|    9.0|   12|
+-------+-----+

+-------------------+-------+
|             values|buckets|
+-------------------+-------+
|  0.619189370225301|    4.0|
| 0.5096018842446481|    4.0|
| 0.8325259388871524|    8.0|
|0.26322809041172357|    2.0|
| 0.6702867696264135|    5.0|
+-------------------+-------+
only showing top 5 rows



In [48]:
# Example DataFrame
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])

df.show()


+---------+---------+
|category1|category2|
+---------+---------+
|        A|        X|
|        A|        Y|
|        A|        X|
|        B|        Y|
|        B|        X|
|        C|        X|
|        C|        X|
|        C|        Y|
+---------+---------+



In [49]:
df.cube("category1").count().show()
"""
The cube function in PySpark is used to perform data aggregation by generating all possible combinations of grouping columns, similar to how the groupby function works.
However, cube goes a step further by also including aggregations for all subsets of the specified grouping columns, as well as the grand total (aggregation across all data).
This means that if you apply cube on multiple columns, it will generate aggregations for each individual column, all possible pairs, all possible triplets, and so on, along with the overall total.
For example, in the code you provided, df.cube("category1").count().show(), the cube function is applied to the "category1" column.
This will generate counts for each distinct value in "category1" as well as a total count for all categories combined.
"""

+---------+-----+
|category1|count|
+---------+-----+
|        B|    2|
|     null|    8|
|        A|    3|
|        C|    3|
+---------+-----+



In [50]:
# Contingency table
df.crosstab('category1', 'category2').show()
"""
The crosstab function in PySpark is used to compute a contingency table (also known as a cross-tabulation) for two columns of a DataFrame.
 It essentially creates a frequency table that shows the distribution of values in one column, categorized by the values in another column.
In your code, df.crosstab('category1', 'category2').show(), it generates a table that shows how many times each combination of values from "category1" and "category2" appears in your DataFrame.
This allows you to see the relationship or association between those two categorical variables
"""

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  B|  1|  1|
|                  C|  2|  1|
|                  A|  2|  1|
+-------------------+---+---+



In [60]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)
# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))
# Show the DataFrame
df.show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [61]:
from pyspark.sql.functions import col, when

df = df.withColumn("is_multiple_of_3", when(col("random")%3==0, 1).otherwise(0))
df.show()

+---+------+----------------+
| id|random|is_multiple_of_3|
+---+------+----------------+
|  0|     7|               0|
|  1|     6|               1|
|  2|     9|               1|
|  3|     3|               1|
|  4|     7|               0|
|  5|     9|               1|
|  6|     7|               0|
|  7|     3|               1|
|  8|     3|               1|
|  9|     7|               0|
+---+------+----------------+



In [62]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [66]:
pos = [0, 4, 8, 5]

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("index", row_number().over(w) - 1)

df.show()

# Filter the DataFrame based on the specified positions
df_filtered = df.filter(df.index.isin(pos))

df_filtered.show()

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  1|     6|    1|
|  2|     9|    2|
|  3|     3|    3|
|  4|     7|    4|
|  5|     9|    5|
|  6|     7|    6|
|  7|     3|    7|
|  8|     3|    8|
|  9|     7|    9|
+---+------+-----+

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  4|     7|    4|
|  5|     9|    5|
|  8|     3|    8|
+---+------+-----+

